<a href="https://colab.research.google.com/github/PadmarajBhat/Real-Time-Analytics-on-Hadoop-Notes/blob/master/pyspark_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 131kB/s 
     |████████████████████████████████| 204kB 37.8MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=833304e0360467d406a8228d299f1507184cda6bfb04bc1ec4e5ba5c52aba7eb
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist



In [0]:
(train_x,train_y), (test_x, test_y) = mnist.load_data()

In [8]:
train_x.shape,train_y.shape, test_x.shape, test_y.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [20]:
pd_train= pd.DataFrame(train_x.reshape(60000,784))
pd_test = pd.DataFrame(test_x.reshape(10000,784))
pd_train.shape,pd_test.shape


((60000, 784), (10000, 784))

In [31]:
pd_train.values *.1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Note that we had to multiply .1 to convert the data to float else not accepted in the tensorflow.

In [32]:
model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(28,28))
    tf.keras.layers.Flatten()
    ,tf.keras.layers.Dense(256,activation="relu")
    ,tf.keras.layers.Dense(128,activation="relu")
    ,tf.keras.layers.Dense(10,activation="softmax")
])

model.compile(optimizer="Adam"
              , loss="sparse_categorical_crossentropy"
              , metrics=['accuracy']
             )

#model.fit(train,train_y, epochs=6)
model.fit(pd_train.values *.1,train_y,epochs=6)
model.evaluate(test_x,test_y)

Epoch 1/6
60000/60000 [==============================] - 6s 99us/sample - loss: 0.2867 - acc: 0.9256
Epoch 2/6
60000/60000 [==============================] - 6s 100us/sample - loss: 0.1264 - acc: 0.9624
Epoch 3/6
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0967 - acc: 0.9718
Epoch 4/6
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0790 - acc: 0.9758
Epoch 5/6
60000/60000 [==============================] - 6s 98us/sample - loss: 0.0692 - acc: 0.9791
Epoch 6/6
10000/10000 [==============================] - 0s 44us/sample - loss: 0.8385 - acc: 0.9752


[0.8385225829925507, 0.9752]

Tensorflow is being fed with panda which indicates that entire data is loaded. This is not a good fit for Big Data case. We need the tensorflow to work on the RDD or Df rather than panda df. Hence spark read --> convert to panda --> tensorflow pipeline fails at the second stage. For this we have https://github.com/maxpumperla/elephas


Elephas: Take spark dataframe and trains it for the keras model. But the example does not talk about tensorflow keras but keras independent version. Not sure if tensorflow keras supports it.

Why we have to worry about tf.keras or keras as independent one ? 
  - tensorflow official supports keras and hence product would have enhancements
  - tensorflow has distributed stratergy to support distributed training. elephas has it too but may be tensorflow is better tested one becuase of larger community of dev and tester.
  - tensorflow has official distribution on hadoop and hence better than smaller community like elephas
